# Pipelines to evaluate fine-mapping benchmarks

This document contains several workflows to extract results from fine-mapping benchmarks and make plots for (mostly) SuSiE manuscript.

In [ ]:
[global]
parameter: cwd = path('~/GIT/lab-dsc/dsc-finemap')
parameter: version = '20190610'
# set to 0 to use estimated prior, or another number to use specified prior
# as configured in the DSC
parameter: susie_prior = [0.1, 0.0]
# null weight options to evaluate
parameter: null_weight = [0.0]
parameter: output_dir = path(f'{cwd:a}/output/susie_paper')

def fmtP(x):
    return str(x).replace(".", "p")

## Query results

In [ ]:
[*_1]
output: f'{output_dir}/result_{version}.rds'
R: expand = '${ }', workdir = cwd
    dap_out = dscrutils::dscquery(${output_dir:br},
                        target = "small_data.dataset lm_pve02.pve lm_pve02.n_signal score_dap.total score_dap.valid score_dap.size score_dap.avgr2 score_dap.top score_dap.overlap score_dap.singal_pip score_dap.pip")
    susie_out = dscrutils::dscquery(${output_dir:br},
                        target = "small_data.dataset lm_pve02.pve lm_pve02.n_signal susie.prior_var susie.null_weight score_susie.total score_susie.valid score_susie.size score_susie.purity score_susie.top score_susie.objective score_susie.converged score_susie.overlap score_susie.signal_pip score_susie.pip")
    caviar_out = dscrutils::dscquery(${output_dir:br},
                        target = "small_data.dataset lm_pve02.pve lm_pve02.n_signal caviar.args score_caviar.total score_caviar.valid score_caviar.size score_caviar.signal_pip score_caviar.pip")
    finemap_out = dscrutils::dscquery(${output_dir:br},
                        target = "small_data.dataset lm_pve02.pve lm_pve02.n_signal finemap.args score_finemap.total score_finemap.valid score_finemap.size score_finemap.signal_pip score_finemap.pip")
    saveRDS(list(dap=dap_out, susie=susie_out, caviar=caviar_out, finemap=finemap_out), ${_output:r})

## PIP calibration